# Защита персональных данных клиентов

## Описание проекта

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Загрузка данных

Импортируем необходимые библиотеки.

In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

Загрузим исходный датасет и выведем первые пять строк для проверки корректности загрузки.

In [3]:
insurance_df = pd.read_csv('insurance.csv', sep=None, engine='python')
insurance_df.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


Выведем общую информацию о датасете, чтобы проверить наличие пропущенных данных.

In [4]:
insurance_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


Пропусков нет. Можно переходить к следующему пункту.

## Умножение матриц

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц) размера $m * n$

- $y$ — вектор целевого признака размера $m$

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

**Ответ:** качество модели линейной регрессии не изменится.

**Обоснование:**

1. Пусть новая матрица признаков получена умножением на обратимую матрицу $P$ справа, и каждый элемент новой матрицы будет представлять собой линейную комбинацию признаков:

$$
X^* = X P
$$

2. Тогда новые значения весов линейной регрессии:

$$
w^* = ((X^*)^T X^*)^{-1} (X^*)^T y = ((X P)^T X P)^{-1} (X P)^T y
$$

3. Используем свойство $(A B)^T = B^T A^T$

$$
w^* = (P^T X^T X P)^{-1} P^T X^T y
$$

4. Так как матрицы $P$ и $X^T X$ невырожденные и квадратные, то $X^T X P$ - тоже невырожденная и обратимая матрица, а значит, мы можем воспользоваться свойством $(A B)^{-1} = B^{-1} A^{-1}$ и сочетательным свойством произведения матриц

$$
w^* = (X^T X P)^{-1} (P^T)^{-1} P^T X^T y = (X^T X P)^{-1} E X^T y = (X^T X P)^{-1} X^T y
$$

5. Ещё раз воспользуемся свойством обратной матрицы произведения матриц

$$
w^* = P^{-1} (X^T X)^{-1} X^T y
$$

6. Таким образом

$$
w^* = P^{-1} w
$$

7. Теперь покажем, что качество модели не изменится. Прогнозные значения линейной регрессии после изменения признаков:

$$
a^* = X P w^* = X P P^{-1} w = X w = a
$$

А значит $MSE(a, y) = MSE(a^*, y)$, как и любые другие оценки качества модели. При этом, зная матрицу $P$, можно найти исходные значения признаков, домножив $X^*$ справа на $P^{-1}$.

В рассмотренном варианте мы домножали матрицу признаков на матрицу $P$ справа, рассмотрим случай домножения слева.

1. Пусть новая матрица признаков:

$$
X^* = P X
$$

2. Тогда новые значения весов линейной регрессии:

$$
w^* = ((P X)^T P X)^{-1} (P X)^T y
$$

3. Используем свойство $(A B)^T = B^T A^T$

$$
w^* = (X^T P^T P X)^{-1} X^T P^T y
$$

Видно, что здесь уже не получится раскрыть обратную матрицу произведения, как это мы проделали в первом варианте решения. Однако, так как $X$ - прямоугольная матрица, то возможно её сингулярное разложение $X = U \Sigma V^T$ в поле вещественных чисел, где $\Sigma_{m,n}$ - матрица размера {\displaystyle m\times n}m\times n с неотрицательными элементами, у которой элементы, лежащие на главной диагонали — это сингулярные числа (а все элементы, не лежащие на главной диагонали, являются нулевыми), а $U_{m,m}$ и $V_{n,n}$ - это две унитарные матрицы, состоящие из левых и правых сингулярных векторов соответственно, причём для случая вещественных чисел ещё и ортогональные. Тогда выражение выше преобразуется в следующее:

$$
w^* = ((U \Sigma V^T)^T P^T P U \Sigma V^T)^{-1} (U \Sigma V^T)^T P^T y
$$

4. Раскроем траспонирование согласно известному свойству:

$$
w^* = (V \Sigma^T U^T P^T P U \Sigma V^T)^{-1} V \Sigma^T U^T P^T y
$$

5. Пусть $P = U^T$, тогда выражение упрощается ввиду $U U^T = U U^{-1} = E$:

$$
w^* = (V \Sigma^T \Sigma V^T)^{-1} V \Sigma^T y
$$

6. Теперь раскроем скобки согласно свойству обратной матрицы произведения:

$$
w^* = (V^T)^{-1} (\Sigma^T \Sigma)^{-1} V^{-1} V \Sigma^T y = 
V (\Sigma^T \Sigma)^{-1} \Sigma^T y
$$

7. Раскроем веса линейной регрессии без изменения признаков:

$$
w = (V \Sigma^T U^T U \Sigma V^T)^{-1} V \Sigma^T U^T y = 
V (\Sigma^T \Sigma)^{-1} \Sigma^T U^T y
$$

8. Тогда предсказания модели до и после изменения признаков:

$$
a = U \Sigma V^T V (\Sigma^T \Sigma)^{-1} \Sigma^T U^T y = U \Sigma (\Sigma^T \Sigma)^{-1} \Sigma^T U^T y
$$

$$
a^* = U^T U \Sigma V^T V (\Sigma^T \Sigma)^{-1} \Sigma^T U^T y = \Sigma (\Sigma^T \Sigma)^{-1} \Sigma^T y
$$

Как видно, предсказания различаются в данном частном случае, а значит, и в общем случае ошибки в одних и тех же точках будут разные и качество скорее всего будет другим.

## Алгоритм преобразования

**Алгоритм**

1. Добавить к признакам единичный вектор.
2. Создать матрицу случайных чисел размера, который требуется для корректного перемножения.
3. Проверить, что определитель данной матрицы не равен нулю (и матрица является обратимой). Если равен нулю, то повторить пункт 2.
4. Перемножить исходную матрицу признаков и матрицу шифрования.

**Обоснование**

При шифровании перемножением справа качество не изменится в силу доказательства выше. В случае с перемножением слева нужно проверить на практике, что скорее всего качество ухудшится.

## Проверка алгоритма

Напишем функцию-шифратор согласно изложенному алгоритму.

In [5]:
def encrypt(features, multiply='right'):
    """
    Функция зашифровывает исходные значения признаков перемножением с матрицей случайных чисел.  
    Параметр 'multiply' позволяет выбирать умножение слева или справа ('right', 'left').
    
    Возвращает зашифрованные признаки.
    
    """
    rows = features.shape[0]
    cols = features.shape[1]
    
    B = np.ones((rows, 1))
    if multiply == 'right':
        len_P = cols + 1
    elif multiply == 'left':
        len_P = rows
    else:
        print('Ошибка: параметр "multiply" получил неизвестное значение!')
        return None
    
    P = np.random.randn(len_P, len_P)
    det_P = np.linalg.det(P)
    
    while abs(det_P) < 10**(-6):
        P = np.random.randn(len_P, len_P)
        det_P = np.linalg.det(P)
    
    features_array = np.array(features)
    features_array = np.concatenate((features_array, B), axis=1)
    
    if multiply == 'right':
        new_features = features_array.dot(P)
    elif multiply == 'left':
        new_features = P.dot(features_array)
    
    return new_features

Разобьём исходный датасет на признаки-регрессоры и целевой признак. Зашифруем двумя способами исходные признаки и добавим к исходным единичный вектор.

In [6]:
B = np.ones((insurance_df.shape[0], 1))
features = np.array(insurance_df.drop(columns=['Страховые выплаты']))

features_encrypted_right = encrypt(features)
features_encrypted_left = encrypt(features, multiply='left')
features = np.concatenate((features, B), axis=1)

target = insurance_df['Страховые выплаты']

features.shape, features_encrypted_right.shape, features_encrypted_left.shape, target.shape

((5000, 5), (5000, 5), (5000, 5), (5000,))

Обучим линейную регрессию на всех объектах датасета и посчитаем коэффициент детерминации для каждого случая.

In [7]:
features_dict = {
    'Исходные признаки': features,
    'Зашифрованные признаки, умножение справа': features_encrypted_right,
    'Зашифрованные признаки, умножение слева': features_encrypted_left
}

r2_list = []

for desc, feat in features_dict.items():
    lr = LinearRegression(fit_intercept=False, normalize=True)
    lr.fit(feat, target)
    predictions = lr.predict(feat)
    r2 = r2_score(target, predictions)
    r2_list.append(r2)

Выведем полученные значения коэффициента детерминации.

In [8]:
for desc, r2 in zip(list(features_dict.keys()), r2_list):
    print('{}, R2 = {:.6f}'.format(desc, r2))

Исходные признаки, R2 = 0.424946
Зашифрованные признаки, умножение справа, R2 = 0.424946
Зашифрованные признаки, умножение слева, R2 = -0.100936


Качество при шифровании умножением справа не изменилось. При шифровании слева качество заметно ухудшилось, что подтвердило гипотезу и показало неприменимость данного варианта.

# Вывод

Для шифрования данных клиентов можно использовать умножение матрицы исходных признаков на матрицу случайных чисел справа, при этом качество линейной модели не изменится, что подтверждено теоретическим доказательством и практической проверкой.

## Чек-лист проверки

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные загружены
- [x]  Выполнен шаг 2: получен ответ на вопрос об умножении матриц
    - [x]  Указан правильный вариант ответа
    - [x]  Вариант обоснован
- [x]  Выполнен шаг 3: предложен алгоритм преобразования
    - [x]  Алгоритм описан
    - [x]  Алгоритм обоснован
- [x]  Выполнен шаг 4: алгоритм проверен
    - [x]  Алгоритм реализован
    - [x]  Проведено сравнение качества моделей до и после преобразования